# Formatting Annotations into Test Data Format  

This script is to automatically format the transriptions with their annotations into the proper format that RASA accepts as training data.

## Importing Completed Transcripts

In [51]:
# Array of names of transcripts with the "Complete" tag
completedTranscripts = []

# Going through all 115 transcript files
for i in range(1, 116):
    # Opening transcript files
    fileName = "transcript_" + str(i) + ".txt"
    file = open("./data/transcripts/" + fileName, "r")

    # Parsing away the name
    file.readline()

    # Checking for "Complete" tag and appending file name if completed
    complete = file.readline()[8:16]
    if (complete == "Complete"):
        completedTranscripts.append(fileName)

    # Closing files
    file.close()

# Sanity Check
print(completedTranscripts)

['transcript_10.txt', 'transcript_22.txt', 'transcript_44.txt', 'transcript_56.txt']


## Filtering and Sorting Different Annotations  

**Possible Annotations:**
- [question] 
- [offer-answer()]
- [options]
- [offer-to-answer]
- [check-answer]
- [agreement]
- [ask-agreement]
- [chit-chat]
- [accept-answer]
- [final-answer()]
- [confirm-agreement]
- [confirm-final-answer()]
- [reject-option()]
- [reject-option-agreement()]

In [52]:
# Without Input
question = []
options = []
offer_to_answer = []
check_answer = []
agreement = []
ask_agreement = []
chit_chat = []
accept_answer = []
confirm_agreement = []

# With Input
offer_answer = []
final_answer = []
confirm_final_answer = []
reject_option = []
reject_option_agreement = []

for fileName in completedTranscripts:
    file = open("./data/transcripts/" + fileName, "r")
    for i in range(0,15):
        file.readline()

    for line in file:
        if (line[0] == 'S'):
            line = line[3:]
        elif (line[0] == 'U'):
            line = line[4:]

        annotation = line[line.find('['):line.find(']') + 1].replace(" ", "")[1:-1]

        if (annotation == "question"):
            question.append(line[:line.find("[") - 1])
        elif (annotation == "options"):
            options.append(line[:line.find("[") - 1])
        elif (annotation == "offer-to-answer"):
            offer_to_answer.append(line[:line.find("[") - 1])
        elif (annotation == "check-answer"):
            check_answer.append(line[:line.find("[") - 1])
        elif (annotation == "agreement"):
            agreement.append(line[:line.find("[") - 1])
        elif (annotation == "ask-agreement"):
            ask_agreement.append(line[:line.find("[") - 1])
        elif (annotation == "chit-chat"):
            chit_chat.append(line[:line.find("[") - 1])
        elif (annotation == "accept-answer"):
            accept_answer.append(line[:line.find("[") - 1])
        elif (annotation == "confirm-agreement"):
            confirm_agreement.append(line[:line.find("[") - 1])
        else:
            annotationName = annotation[:annotation.find('(')]
            if (annotationName == "offer-answer"):
                offer_answer.append([line[:line.find("[") - 1], annotation[annotation.find('(') + 1:annotation.find(')')]])
            elif (annotationName == "final-answer"):
                final_answer.append([line[:line.find("[") - 1], annotation[annotation.find('(') + 1:annotation.find(')')]])
            elif (annotationName == "confirm-final-answer"):
                confirm_final_answer.append([line[:line.find("[") - 1], annotation[annotation.find('(') + 1:annotation.find(')')]])
            elif (annotationName == "reject-option"):
                reject_option.append([line[:line.find("[") - 1], annotation[annotation.find('(') + 1:annotation.find(')')]])
            elif (annotationName == "reject-option-agreement"):
                reject_option_agreement.append([line[:line.find("[") - 1], annotation[annotation.find('(') + 1:annotation.find(')')]])
    file.close()


    

['Which of these cities is most associated with Robin Hood?', 'How many girls are there in the group Girls Aloud', 'Which of these politicians is the former MP for Chesterfield', 'Which of these colors does not appear on the national flag of Brazil?', "question number one is on indepence. During which traditional event might you hear the shout it's behind you", 'which of these items is most likely to be used for washing in the bathroom?', 'someone given an unwelcome responsibility is said to be left holding what?', 'Alan Whicker is best known for presenting TV programs on what subjects?', 'in 2005, you know this, Malcolm Glazer took control of which British football club?', 'what kind of drink is an americano?', 'the Australian Joan Sutherland was a leading figure in which branch of the arts?', 'Vermicide is a substance used to kill which creatures?', 'Vermicide', 'is a substance to kill which creatures?', 'which of these is a type of fabric?', 'what is the capital of the Italian regio